In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
# API_KEY = ""

In [ ]:
import cv2
import base64
from anthropic import Anthropic
import json
import re

num_images = 5

def analyze_video_frames_unique(video_path, api_key):
    client = Anthropic(api_key=api_key)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // num_images

    frames_base64 = []
    for i in range(num_images):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            _, buffer = cv2.imencode('.jpg', frame)
            frames_base64.append(base64.b64encode(buffer).decode('utf-8'))

    cap.release()

    messages = [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """Return ONLY a JSON array with exactly 2 items containing unique main objects from these frames. Rules:
                1. Format: ["Object1", "Object2"] or ["Object1", null]
                2. Maximum 2 unique objects
                3. No explanation text, just the JSON array"""
            }
        ]
    }]

    for img_base64 in frames_base64:
        messages[0]["content"].append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": img_base64
            }
        })

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=150,
        messages=messages
    )

    # Extract JSON array using regex
    json_match = re.search(r'\[.*?\]', response.content[0].text)
    if json_match:
        return json.loads(json_match.group())
    return [None, None]

api_key = API_KEY
video_path = "/content/drive/MyDrive/Genuin Assignment/single_class_ip/o9tzeZckDgE.mp4"
unique_objects = analyze_video_frames_unique(video_path, api_key)
print(unique_objects)

['Person', 'Surfboard']
